In [1]:
!pip3 install vaderSentiment
!pip3 install flair
!pip3 install spacytextblob

     |████████████████████████████████| 125 kB 4.3 MB/s 
     |████████████████████████████████| 319 kB 4.3 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 1.2 MB 49.9 MB/s 
     |████████████████████████████████| 788 kB 59.4 MB/s 
     |████████████████████████████████| 981 kB 52.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 5.4 MB/s 
     |████████████████████████████████| 19.7 MB 1.5 MB/s 
     |████████████████████████████████| 2.8 MB 58.1 MB/s 
     |████████████████████████████████| 62 kB 840 kB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 636 kB 32.7 MB/s 
     |████████████████████████████████| 895 kB 74.7 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9

In [24]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 13.6 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
!pip3 install spacytextblob

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
cd MyDrive/Omdena\ Colombia\ Chapter

/gdrive/MyDrive/Omdena Colombia Chapter


In [4]:
import pandas as pd
import re
import string
from google.colab import files

In [5]:
data = pd.read_csv("Corrected_Final_All.csv")

In [6]:
data_english = data[data['lang']=='en']

In [7]:
data_english.isna().sum()

Unnamed: 0                               0
created_at                               0
id_str                                1039
conversation_id_str                   1039
full_text                                0
twitter_lang                          1039
favorited                             1039
retweeted                             1039
retweet_count                         1039
favorite_count                        1039
reply_count                           1039
quote_count                           1039
quoted_status_id_str                  4005
quoted_status_short_url               5470
quoted_status_expand_url              5470
user_id_str                           1039
user_name                             1039
user_full_name                        1040
user_verified                         1039
in_reply_to_status_id_str             4445
in_reply_to_user_id_str               4254
hashtags                              1040
mentions                              3351
urls       

In [8]:
#To preprocess list present as string, specially for column "preprocessed_tweets_without_hashtags"
import ast
x = '[ "I","am","not","here"]'
x  = ast.literal_eval(x)
print(x) ##returns as list of strings ['I', 'am', 'not', 'here']
print(" ".join(x)) ##returns as concatenated string separated by spaces --> I am not here

['I', 'am', 'not', 'here']
I am not here


change name of column according to what column you are using for sentiment analysis

In [9]:
#TextBlob
from textblob import TextBlob

score_textBlob = []

for i in range(data_english.shape[0]):
  testimonial = TextBlob(data['full_text'].iloc[i])
  score_textBlob.append(testimonial.sentiment)

data_english['sentiment_full_text_textBlob'] = score_textBlob

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
score_VADER = []

for i in range(data_english.shape[0]):
  vs = analyzer.polarity_scores(data['full_text'].iloc[i])
  score_VADER.append(str(vs))

data_english['sentiment_full_text_vader'] = score_VADER

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [28]:
#Flair
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
score_Flair = []

for i in range(data_english.shape[0]):
  sentence = Sentence(data['full_text'].iloc[i])
  classifier.predict(sentence)
  score_Flair.append(sentence.labels)

data_english['sentiment_full_text_flair'] = score_Flair

2021-09-21 12:27:32,225 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [29]:
#SpacyTextBlob
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
score_spacy = []

for i in range(data_english.shape[0]):
  text = data['full_text'].iloc[i]
  doc = nlp(text)
  score_spacy.append((doc._.polarity,doc._.subjectivity,doc._.assessments))

data_english['sentiment_full_text_spacytextblob'] = score_spacy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
data_english.tail(20)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_full_text_textBlob,sentiment_full_text_vader,sentiment_full_text_flair,sentiment_full_text_spacytextblob
22140,22140,2021-06-08 21:32:42,NaN,NaN,"There's one called Exigimos by Doctor Krapula,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08,"['there', 'is', 'one', 'called', 'exigir', 'by...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['one', 'call', 'exigimos', 'doctor', 'krapula...","(0.0, 0.0)","{'neg': 0.0, 'neu': 0.93, 'pos': 0.07, 'compou...",[POSITIVE (0.9939)],"(0.0, 0.0, [])"
22141,22141,2021-06-08 22:03:59,NaN,NaN,[this one?](https://imgur.com/gallery/ZxPTNFX),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08,"['this', 'one']",[''],[],NaN,Reddit,en,1.0,NaN,en,['one'],"(0.0, 0.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",[POSITIVE (0.9073)],"(0.0, 0.0, [])"
22142,22142,2021-06-08 22:24:59,NaN,NaN,Cool!!! Thanks!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08,"['cool', 'thanks']",[''],[],NaN,Reddit,en,1.0,NaN,en,"['cool', 'thanks']","(0.0, 1.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",[POSITIVE (0.986)],"(0.0, 1.0, [(['final'], 0.0, 1.0, None)])"
22143,22143,2021-06-10 2:22:30,NaN,NaN,Listen my spotify revolution playlist: [https:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-10,"['listen', 'my', 'spotify', 'revolution', 'pla...",[''],[],NaN,Reddit,en,2.0,NaN,en,"['listen', 'spotify', 'revolution', 'playlist'...","(0.0, 0.0)","{'neg': 0.132, 'neu': 0.868, 'pos': 0.0, 'comp...",[NEGATIVE (0.9906)],"(0.0, 0.0, [])"
22144,22144,2021-06-13 23:31:16,NaN,NaN,based,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-13,['based'],[''],[],NaN,Reddit,en,1.0,NaN,en,['base'],"(0.6, 0.9)","{'neg': 0.059, 'neu': 0.865, 'pos': 0.076, 'co...",[POSITIVE (0.9666)],"(0.6, 0.9, [(['popular'], 0.6, 0.9, None)])"
22145,22145,2021-06-29 1:15:49,NaN,NaN,>Ya hay r/LatinoPeopleTwitter\n\nMás bien r/Gr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-29,"['rlatinopeopletwitter', 'mas', 'bien', 'rgrin...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['ya', 'hay', 'rlatinopeopletwitter', 'más', '...","(0.03333333333333333, 0.06666666666666667)","{'neg': 0.05, 'neu': 0.95, 'pos': 0.0, 'compou...",[NEGATIVE (0.9374)],"(0.03333333333333333, 0.06666666666666667, [([..."
22146,22146,2021-06-30 13:22:29,NaN,NaN,Smells like massive debt and the origins of a ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-30,"['smells', 'like', 'massive', 'debt', 'and', '...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['smell', 'like', 'massive', 'debt', 'origin',...","(0.1, 0.1)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",[POSITIVE (0.9363)],"(0.1, 0.1, [(['cultural'], 0.1, 0.1, None)])"
22147,22147,2021-07-06 13:32:21,NaN,NaN,The results are interesting to follow over tim...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-06,"['the', 'results', 'are', 'interesting', 'to',...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['result', 'interest', 'follow', 'time', 'thin...","(0.0, 0.0)","{'neg': 0.155, 'neu': 0.634, 'pos': 0.211, 'co...",[NEGATIVE (0.9514)],"(0.0, 0.0, [])"
22148,22148,2021-07-10 1:02:25,NaN,NaN,But are they mamertos?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-10,"['but', 'are', 'they'

In [ ]:
#Download file with the name "Corrected_Final_All_English_[column_name].csv"
data_english.to_csv("Corrected_Final_All_English_Full_Text.csv", index=False)
files.download("Corrected_Final_All_English_Full_Text.csv")